In [ ]:
from openai import OpenAI

def generate_openai(message): 
    api_key = "..."
    client = OpenAI(api_key = api_key)
    temperature = 0.0
    
    message_log = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": message}
    ]
    response = client.chat.completions.create(
                model='gpt-4.1-mini', 
                messages=message_log,   
                max_tokens=512,
                temperature=temperature,      
            )
    return response.choices[0].message.content




In [2]:
import json

with_dropout = []
without_dropout = []

with open("with_dropout_2000.jsonl", "r") as file:
    for line in file:
        with_dropout.append(json.loads(line))

with open("without_dropout_2000.jsonl", "r") as file:
    for line in file:
        without_dropout.append(json.loads(line))

In [ ]:
import numpy as np

my_styles = []
input_styles = []
gpt_styles = []

my_ai = []
input_ai = []
gpt_ai = []

my_best_styles = []
input_best_styles = []
gpt_best_styles = []

for x in with_dropout:
    if True:#"Subject:" not in x['input_text']:
        cos = x['my_scores']['cosines']
        max_index = np.argmax(cos)
        max_value = np.max(cos)

        my_styles.append(max_value)
        my_best_styles.append(max_index)
        my_ai.append(x['my_scores']['detector_reward'])

        cos = x['input_scores']['cosines']
        max_index = np.argmax(cos)
        max_value = np.max(cos)

        input_styles.append(max_value)
        input_best_styles.append(max_index)
        input_ai.append(x['input_scores']['detector_reward'])

        cos = x['gpt_scores']['cosines']
        max_index = np.argmax(cos)
        max_value = np.max(cos)

        gpt_styles.append(max_value)
        gpt_best_styles.append(max_index)
        gpt_ai.append(x['gpt_scores']['detector_reward'])


np.mean(my_styles), np.mean(input_styles), np.mean(gpt_styles)

In [ ]:
np.mean(my_ai), np.mean(input_ai), np.mean(gpt_ai)

In [ ]:
from collections import Counter
Counter(my_best_styles), Counter(input_best_styles), Counter(gpt_best_styles)

In [ ]:
# I see the bias in case we have word "Subject:" at the beginning

rebuilt = {}
count = 0
for x in with_dropout:
    if "Subject:" not in x['input_text']:
        count+=1
        for k,v in x.items():
            if "score" in k:
                for ik,iv in v.items():
                    if k+"_"+ik in rebuilt:
                        rebuilt[k+"_"+ik].append(iv)
                    else:
                        rebuilt[k+"_"+ik] = [iv]
count

18

In [7]:
import random
from tqdm import tqdm

def gudge(reference, answer_a, answer_b):
    gpt_judge_prompt = f'''Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below.
Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses.
Begin your evaluation by comparing the two responses and provide a short explanation. 
Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. 
Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants.
Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]" if assistant B is better, and "[[C]]" for a tie.

The task agents are solving - they are rewriting the reference text.
The text created by assistant should maintain the same meaning as the reference text.
The text created by assistant should maintain correct grammar. Small mistakes are allowed.

[Reference text]
{reference}
[End of reference text]

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]
[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]'''
    return generate_openai(gpt_judge_prompt)
    
judgements = []
with tqdm(total = len(with_dropout)) as pbar:
    for obj in with_dropout:
        a = obj['rewritten_text_my']
        b = obj['rewritten_text_gpt']
        c = obj['input_text']

        if random.random()>0.5:
            first = a
            second = b
            first_mine = True
        else:
            first = b
            second = a
            first_mine = False

        g = gudge(c, a,b)
        judgements.append({"first_mine": first_mine, "judgement": g})
        pbar.update(1)

100%|██████████| 40/40 [03:27<00:00,  5.18s/it]


In [8]:
judgements_without_dropout = []
with tqdm(total = len(without_dropout)) as pbar:
    for obj in without_dropout:
        a = obj['rewritten_text_my']
        b = obj['rewritten_text_gpt']
        c = obj['input_text']

        if random.random()>0.5:
            first = a
            second = b
            first_mine = True
        else:
            first = b
            second = a
            first_mine = False

        g = gudge(c, first, second)
        judgements_without_dropout.append({"first_mine": first_mine, "judgement": g})
        pbar.update(1)

100%|██████████| 40/40 [03:23<00:00,  5.09s/it]


In [ ]:
# judgements for with dropout samples
mine = 0
gpt = 0
tie = 0
for x in judgements:
    if x['first_mine']:
        if "[[A]]" in x['judgement']:
            mine+=1
        elif "[[B]]" in x['judgement']:
            gpt+=1
        else:
            tie+=1
    else:
        if "[[A]]" in x['judgement']:
            gpt+=1
        elif "[[B]]" in x['judgement']:
            mine+=1
        else:
            tie+=1
mine, gpt, tie

(16, 23, 1)

In [ ]:
# judgements for without dropout samples
mine = 0
gpt = 0
tie = 0
for x in judgements_without_dropout:
    if x['first_mine']:
        if "[[A]]" in x['judgement']:
            mine+=1
        elif "[[B]]" in x['judgement']:
            gpt+=1
        else:
            tie+=1
    else:
        if "[[A]]" in x['judgement']:
            gpt+=1
        elif "[[B]]" in x['judgement']:
            mine+=1
        else:
            tie+=1
mine, gpt, tie

(0, 40, 0)